In [1]:
import os 
import math 
import numpy as np 
import pandas as pd 
from pylab import plt,mpl
plt.style.use('seaborn')
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.float_format', '{:.4f}'.format)
np.set_printoptions(suppress = True, precision = 4)
os.environ['PYTHONHASHSEED'] = '0'


In [2]:
url = 'http://hilpisch.com/aiif_eikon_id_eur_usd.csv'
symbol = 'EUR='

In [3]:
data = pd.DataFrame(pd.read_csv(url, index_col = 0, 
                               parse_dates = True).dropna()['CLOSE'])

In [4]:
data.columns = [symbol]

In [5]:
data = data.resample('5min', label = 'right').last().ffill()

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26486 entries, 2019-10-01 00:05:00 to 2019-12-31 23:10:00
Freq: 5T
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   EUR=    26486 non-null  float64
dtypes: float64(1)
memory usage: 413.8 KB


In [8]:
lags = 5

lags = 5
def add_lags(data, symbol, lags, window = 20):
    cols = []
    df = data.copy()
    df.dropna(inplace = True)
    df['r'] = np.log(df / df.shift(1))
    df['sma'] = df[symbol].rolling(window).mean()
    df['min'] = df[symbol].rolling(window).min()
    df['max'] = df[symbol].rolling(window).max()
    df['mom'] = df['r'].rolling(window).mean()
    df['vol'] = df['r'].rolling(window).std()
    df.dropna(inplace = True)
    df['d'] = np.where(df['r']>0,1,0)
    features = [symbol, 'r', 'd', 'sma', 'min', 'max', 'mom', 'vol']
    for f in features:
        for lag in range(1, lags+1):
            col = f'{f}_lag_{lag}'
            df[col] = df[f].shift(lag)
            cols.append(col)
    df.dropna(inplace = True)
    return df, cols

data, cols = add_lags(data, symbol, lags, window = 20)




ValueError: Columns must be same length as key